# Bag-Of-Words Classifier

Import necessary dependencies from scikit. Use [pandas](https://pandas.pydata.org/) for file extraction. 

In [111]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

fileName = 'wikipedia_300/wikipedia_300.csv'
col = ['Text', 'Category']

# Read the dataset and split text and category
df = pd.read_csv(fileName, header=0, sep=',', names=col)
X_train, X_test, y_train, y_test = train_test_split(df['Text'], df['Category'], random_state = 0, test_size=0.2)

# Init the countvectorizer and convert the text/documents to a matrix of token counts
# I.e create a bag of words
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

240


### Using Multinominal Naive Bayes

In [86]:
NB_model = MultinomialNB().fit(X_train_counts, y_train)

# Check if model is able to predict
print(NB_model.predict(count_vect.transform(["Script for games are good with java "])))

['Games']


### Using Support Vector Machines

In [88]:
SVM_model = LinearSVC().fit(X_train_counts, y_train)

# Check if model is able to predict
print(SVM_model.predict(count_vect.transform(["Script for games are good with java"])))

['Games']


### Comparing Accuracy

Here we are comparing the accuracy score between the Naive Bayes method and the SVM method. We can see that the SVM has a higher accuracy score than the Naive Bayes method on classifing the text

In [138]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import KFold, cross_val_score

X_test_counts = count_vect.transform(X_test)
k_fold = KFold(n_splits=10)

NB_predictions = NB_model.predict(X_test_counts)
SVM_predictions = SVM_model.predict(X_test_counts)

print('/====== Multinominal Naive Bayes Classifier ======/')
print(classification_report(y_test, NB_predictions))
print(f'Accuracy Score: {round(accuracy_score(y_test, NB_predictions) * 100, 2)}%')
print('Confusion Matrix:')
print(confusion_matrix(y_test, NB_predictions))
print(k_fold)

print()
print()

print('/====== Support Vector Machine Classifier ======/')
print(classification_report(y_test, SVM_predictions))
print(f'Accuracy Score: {round(accuracy_score(y_test, SVM_predictions) * 100, 2)}%')
print('Confusion Matrix:')
print(confusion_matrix(y_test, SVM_predictions))

/====== Multinominal Naive Bayes Classifier ======/
              precision    recall  f1-score   support

       Games       0.93      0.90      0.91        29
 Programming       0.91      0.94      0.92        31

   micro avg       0.92      0.92      0.92        60
   macro avg       0.92      0.92      0.92        60
weighted avg       0.92      0.92      0.92        60

Accuracy Score: 91.67%
Confusion Matrix:
[[26  3]
 [ 2 29]]
KFold(n_splits=10, random_state=None, shuffle=False)


/====== Support Vector Machine Classifier ======/
              precision    recall  f1-score   support

       Games       0.96      0.90      0.93        29
 Programming       0.91      0.97      0.94        31

   micro avg       0.93      0.93      0.93        60
   macro avg       0.94      0.93      0.93        60
weighted avg       0.94      0.93      0.93        60

Accuracy Score: 93.33%
Confusion Matrix:
[[26  3]
 [ 1 30]]


### Comparing Accuracy using Cross Validation

By measuring the accuracy using cross validation, we get a completely different score. Here we are splitting the text documents into 10 folds (i.e. 10 groups), and checking the accuracy score for each fold. By measuring the mean accuracy score, we can see that the Naive Bayes method has a higher accuracy than SVM, but also has a higher spectrum of values.

In [139]:
print('/====== Multinominal Naive Bayes Classifier ======/')
scores = cross_val_score(NB_model, count_vect.transform(df['Text']), df['Category'], cv=10)
print(scores)
print(f'Accuracy mean: {round(scores.mean()*100, 2)}% (+/- {round(scores.std()*2*100, 2)}%)')

print()
print('/====== Support Vector Machine Classifier ======/')
scores = cross_val_score(SVM_model, count_vect.transform(df['Text']), df['Category'], cv=10)
print(scores)
print(f'Accuracy mean: {round(scores.mean()*100, 2)}% (+/- {round(scores.std()*2*100, 2)}%)')

/====== Multinominal Naive Bayes Classifier ======/
[0.96666667 0.96666667 0.9        0.96666667 0.93333333 1.
 0.9        0.96666667 1.         0.93333333]
Accuracy mean: 95.33% (+/- 6.8%)

/====== Support Vector Machine Classifier ======/
[0.93333333 0.93333333 0.9        0.93333333 0.86666667 0.9
 0.93333333 0.93333333 0.93333333 0.9       ]
Accuracy mean: 91.67% (+/- 4.47%)


### Using TF-IDF Converter